In [6]:
import pandas as pd
import numpy as np

print("--- 피처 엔지니어링 스크립트 시작 ---")

# ==============================================================================
# 1. 데이터 불러오기
# ==============================================================================
try:
    # process.py에서 저장한 merged_data.csv 파일을 불러옵니다.
    df = pd.read_csv("./merged_data.csv")
    print("✅ Step 1: 'merged_data.csv' 로딩 성공!")
    print(f"  - 초기 데이터 형태: {df.shape}")
except FileNotFoundError:
    print("❌ 오류: 'merged_data.csv' 파일을 찾을 수 없습니다. process.py를 먼저 실행해주세요.")
    # 파일이 없으면 스크립트를 더 이상 진행하지 않습니다.
    exit()

--- 피처 엔지니어링 스크립트 시작 ---
✅ Step 1: 'merged_data.csv' 로딩 성공!
  - 초기 데이터 형태: (86590, 39)


In [7]:
# ==============================================================================
# 2. 특수 값(-999999.9)을 결측치(NaN)로 변환
# ==============================================================================
# 실수형(float) 컬럼에만 특수 값이 존재하므로 해당 컬럼들을 선택합니다.
float_cols = df.select_dtypes(include=['float64']).columns
df[float_cols] = df[float_cols].replace(-999999.9, np.nan)
print("✅ Step 2: 특수 값(-999999.9)을 결측치(NaN)로 변환 완료!")


✅ Step 2: 특수 값(-999999.9)을 결측치(NaN)로 변환 완료!


In [8]:
# ==============================================================================
# 3. '생존 기간' 등 시간 기반 파생 변수 생성
# ==============================================================================
# 개설일(ARE_D)과 폐업일(MCT_ME_D)을 날짜 형식(datetime)으로 변환합니다.
# format='%Y%m%d'는 'YYYYMMDD' 형식의 숫자를 날짜로 인식하라는 의미입니다.
# errors='coerce'는 변환 중 오류가 발생하면 해당 값을 NaT(Not a Time)으로 처리합니다.
df['ARE_DT'] = pd.to_datetime(df['ARE_D'], format='%Y%m%d', errors='coerce')
df['MCT_ME_DT'] = pd.to_datetime(df['MCT_ME_D'], format='%Y%m%d', errors='coerce')

# 폐업하지 않은 가게(MCT_ME_DT가 NaT인 경우)의 폐업일은
# 데이터의 마지막 시점인 2024년 12월 31일로 가정하여 채워줍니다.
LATEST_DATE = pd.to_datetime('20241231', format='%Y%m%d')
df['MCT_ME_DT'] = df['MCT_ME_DT'].fillna(LATEST_DATE)

# 가게의 '총 운영일수'를 계산하여 새로운 컬럼 'SURVIVAL_DAYS'를 만듭니다.
df['SURVIVAL_DAYS'] = (df['MCT_ME_DT'] - df['ARE_DT']).dt.days

print("✅ Step 3: 'SURVIVAL_DAYS' 파생 변수 생성 완료!")



✅ Step 3: 'SURVIVAL_DAYS' 파생 변수 생성 완료!


In [9]:
# ==============================================================================
# 4. 순서형(Ordinal) 변수를 숫자(Rank)로 변환
# ==============================================================================
# 변환할 컬럼 목록을 정의합니다. 데이터 명세서에 나온 6개 구간 컬럼들입니다.
ordinal_cols = [
    'MCT_OPE_MS_CN', # 가맹점 운영개월수 구간
    'RC_M1_SAA',       # 매출금액 구간
    'RC_M1_TO_UE_CT',  # 매출건수 구간
    'RC_M1_UE_CUS_CN', # 이용고객수 구간
    'RC_M1_AV_NP_AT',  # 건당결제액 구간
]

# 변환에 사용할 매핑(mapping) 딕셔너리를 정의합니다.
# 데이터 명세서에 따라 숫자가 낮을수록 상위 구간을 의미합니다.
ranking_map = {
    '1_상위1구간': 1,
    '2_10-25%': 2,
    '3_25-50%': 3,
    '4_50-75%': 4,
    '5_75-90%': 5,
    '6_90%초과(하위 10% 이하)': 6
}

# for 루프를 사용해 목록에 있는 모든 컬럼을 한 번에 변환합니다.
# 새로운 컬럼 이름은 기존 이름 뒤에 '_RANK'를 붙여 만듭니다.
for col in ordinal_cols:
    df[col + '_RANK'] = df[col].map(ranking_map)

# APV_CE_RAT 컬럼은 구간 이름이 약간 다르므로 별도로 처리합니다.
# (만약 이 컬럼의 구간 이름이 같다면 위 목록에 포함시켜도 됩니다.)
apv_ce_rat_map = {
    '1_상위1구간': 1,
    '2_상위2구간': 2,
    '3_상위3구간': 3,
    '4_상위4구간': 4,
    '5_상위5구간': 5
}
df['APV_CE_RAT_RANK'] = df['APV_CE_RAT'].map(apv_ce_rat_map)


print("✅ Step 4: 모든 순서형 변수를 순위(Rank)로 변환 완료!")



✅ Step 4: 모든 순서형 변수를 순위(Rank)로 변환 완료!


In [10]:
# ==============================================================================
# 5. 최종 데이터 확인 및 저장
# ==============================================================================
print("\n--- 최종 데이터 정보 ---")
# 데이터의 기본 정보와 결측치 현황을 다시 한번 확인합니다.
df.info()

# 변환이 잘 되었는지 상위 5개 행을 출력해 확인합니다.
print("\n--- 최종 데이터 샘플 (상위 5개) ---")
# 새로 생성된 컬럼들을 위주로 확인합니다.
new_cols = ['SURVIVAL_DAYS'] + [col + '_RANK' for col in ordinal_cols] + ['APV_CE_RAT_RANK']
print(df[new_cols].head())

# 최종 결과물을 새로운 CSV 파일로 저장합니다.
output_path = "./featured_data.csv"
# index=False: 불필요한 인덱스 컬럼이 저장되지 않도록 합니다.
# encoding='utf-8-sig': 엑셀에서 열 때 한글이 깨지지 않도록 합니다.
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n🎉 모든 작업 완료! 최종 데이터가 '{output_path}' 파일로 저장되었습니다.")


--- 최종 데이터 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86590 entries, 0 to 86589
Data columns (total 48 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ENCODED_MCT               86590 non-null  object        
 1   TA_YM                     86590 non-null  int64         
 2   MCT_OPE_MS_CN             86590 non-null  object        
 3   RC_M1_SAA                 86590 non-null  object        
 4   RC_M1_TO_UE_CT            86590 non-null  object        
 5   RC_M1_UE_CUS_CN           86590 non-null  object        
 6   RC_M1_AV_NP_AT            86590 non-null  object        
 7   APV_CE_RAT                79958 non-null  object        
 8   DLV_SAA_RAT               29245 non-null  float64       
 9   M1_SME_RY_SAA_RAT         86590 non-null  float64       
 10  M1_SME_RY_CNT_RAT         86590 non-null  float64       
 11  M12_SME_RY_SAA_PCE_RT     86590 non-null  float64       
 12 